- [Linear Regression](#linear-regression)
- [Logistic Regression + MCMC + Moment Matching](#Logistic-Regression-+-MCMC-+-Moment-Matching)
- [Logistic Regression + MCMC](#Logistic-Regression-+-MCMC)
- [Logistic Regression + Search](#Logistic-Regression-+-Search)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, prior=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [0.62945907 1.04030399 1.38943747 0.22187135]
Prior MSE 236.14026486427062 True MSE 0.9295563918386928

Parameter Estimate [ 5.03384098 -2.04242046  1.0458533   4.49254238]
Prior MSE 1.1156529019110772 True MSE 0.9295563918386928

Parameter Estimate [ 5.00171183 -2.10335714  1.05217679  5.25890319]
Prior MSE 0.9547183417513909 True MSE 0.9295563918386928

Parameter Estimate [ 4.94934162 -2.08692361  1.03290793  5.46006742]
Prior MSE 0.8942972424078205 True MSE 0.9295563918386928

Parameter Estimate [ 4.88098409 -2.04564108  0.98936908  5.68943923]
Prior MSE 0.8946674022992412 True MSE 0.9295563918386928

Parameter Estimate [ 4.90174701 -2.06784802  0.98461191  5.80234427]
Prior MSE 0.8618938961334188 True MSE 0.9295563918386928

Parameter Estimate [ 4.91712289 -2.07114437  0.9580543   5.9014963 ]
Prior MSE 0.8525450886358604 True MSE 0.9295563918386928

Parameter Estimate [ 4.92216242 -2.06329766  0.95407032  5.88746845]
Prior MSE 0.848502279680264 True MSE 0.9295563

Logistic Regression + MCMC + Moment Matching
---

In [7]:
%%time


@numba.jit(nopython=True, fastmath=True, forceobj=False)
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

fast_p = pp.normal.fast_p # Need to assign here first since numba does not support jitting methods of classes

def likelihood(y, x, w):
    return fast_p(y - sigmoid(np.sum(x * w[:-1]) + w[-1]), mu=0.0, sigma=1.0)





def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(6):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   prior=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=5,
                                   samples=50000,
                                   mixing=5000, 
                                   energy=0.1,
                                   mode="mcmc")

    w_approx = pp.mode(prior)
    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

0.3988777559584158
Parameter Estimate [-0.40529064  1.04209067  0.84287458 -0.81387161]
Prior MSE 0.06547424398971416 True MSE 0.027118421796964155

Parameter Estimate [-0.5393875   0.88756646  1.65490853 -0.78964772]
Prior MSE 0.05636704233753099 True MSE 0.027118421796964155

Parameter Estimate [-0.94657658  0.74845252  1.98554966 -0.6921813 ]
Prior MSE 0.04999063556989479 True MSE 0.027118421796964155

Parameter Estimate [-1.12361844  1.36179791  2.09708833 -1.11518561]
Prior MSE 0.03905124753589012 True MSE 0.027118421796964155

Parameter Estimate [-1.02130255  1.28607396  2.35524837 -1.19898742]
Prior MSE 0.03962956377991245 True MSE 0.027118421796964155

Parameter Estimate [-1.48019047  1.38324663  2.45710113 -1.11913441]
Prior MSE 0.034888837312293974 True MSE 0.027118421796964155

CPU times: user 19.1 s, sys: 27.9 ms, total: 19.1 s
Wall time: 19.1 s


Logistic Regression + MCMC
---

In [9]:
%%time

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   prior=prior, 
                                   batch=5,
                                   samples=20000,
                                   mixing=18000, 
                                   energy=0.3,
                                   mode="mcmc")

    modes = pp.mode(prior) # modes are sorted in order first is largest

    print("Number of modes", len(modes))
    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 1
Parameter Estimate [-0.40251571  1.63657293  2.64594073 -1.89631666]
Prior MSE 0.043379478686802544 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-1.64726359  1.24265392  4.02096962 -2.03267843]
Prior MSE 0.03318426198006277 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-1.59024907  1.82567696  4.04794259 -2.38693076]
Prior MSE 0.030119804944745553 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-2.14826314  2.05129974  5.11420032 -2.44792898]
Prior MSE 0.03274770047208002 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-2.67868997  2.6145071   5.67260813 -2.71680115]
Prior MSE 0.03226325696568973 True MSE 0.029320377157292802

CPU times: user 39.8 s, sys: 11.9 ms, total: 39.8 s
Wall time: 39.8 s


Logistic Regression + Search
---


In [14]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior
def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 10)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=50,
                                   samples=1000,
                                   energies=0.25,
                                   mode="search",
                                   volume=1000)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    
    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 5
[-2.09769567  1.62888091  2.83550617 -1.66533419]
[-1.70338861  1.4593331   3.57524701 -1.17629273]
[-2.21563849  1.94664001  3.61763174 -1.75253601]
[-1.26829558  1.14523862  3.69190497 -1.82670079]
[-1.85866349  1.49373765  3.82101128 -1.9803491 ]
Parameter Estimate [-2.09769567  1.62888091  2.83550617 -1.66533419]
Prior MSE 0.026647292517380086 True MSE 0.02380137662073885

Number of modes 2
[-1.8877228   1.50536392  2.75388995 -1.56296231]
[-2.05143615  1.62483292  3.29180334 -1.83670573]
Parameter Estimate [-1.8877228   1.50536392  2.75388995 -1.56296231]
Prior MSE 0.026301980245897038 True MSE 0.02380137662073885

Number of modes 2
[-1.94756153  1.64929901  2.81911065 -1.62939674]
[-2.12624048  1.32647163  2.79985965 -1.6073133 ]
Parameter Estimate [-1.94756153  1.64929901  2.81911065 -1.62939674]
Prior MSE 0.02538478344399076 True MSE 0.02380137662073885

Number of modes 1
[-2.08487297  1.65585549  2.86848708 -1.659646  ]
Parameter Estimate [-2.08487297  1.6558